# Travelling Salesman Problem

The Travelling Salesman Problem(TLP) is a classic optimization problem, studied as far back as the 1800s. It involves finding the most efficient route through a collection of cities, visiting each city exactly once.


In [5]:
from operator import attrgetter
import numpy as np
import random

In [136]:
class Individual:
    def __init__(self, chromosome_len):
        # Create an array from 0 to n
        self._chromosome = np.arange(0, chromosome_len)
        self._fitness = -1
    
    @property
    def chromosome(self):
        return self._chromosome

    @chromosome.setter
    def chromosome(self, chromosome):
        self._chromosome = chromosome
    
    @property
    def chromosome_len(self):
        return len(self._chromosome)
    
    def get_gene(self, offset):
        return self._chromosome[offset]
    
    def set_gene(self, offset, gene):
        self._chromosome[offset] = gene
    
    @property
    def fitness(self):
        return self._fitness

    @fitness.setter
    def fitness(self, fitness):
        self._fitness = fitness
    
    def contains_gene(self, gene):
        return gene in self._chromosome
    
    def __str__(self):
        output = ''
        for i, v in enumerate(self._chromosome):
            output += str(v)
        return output
        

In [39]:
class Population:
    def __init__(self, population_size, chromosome_len):
        self._population_fitness = -1
        self._population = [Individual] * population_size
        
        for i in range(population_size):
            self._population[i] = Individual(chromosome_len)
        
    @property
    def individuals(self):
        return self._population
    
    def get_fittest(self, offset):
        self._population = sorted(self._population, key=attrgetter('fitness'), reverse=True)
        return self._population[offset]
    
    @property
    def population_fitness(self):
        return self._population_fitness
    
    @population_fitness.setter
    def population_fitness(self, fitness):
        self._population_fitness = fitness
    
    @property
    def size(self):
        return len(self._population)
    
    def set_individual(self, offset, individual):
        self._population[offset] = individual
    
    def get_individual(self, offset):
        return self._population[offset]
    
    def shuffle(self):
        random.shuffle(self._population)
    
    

In [138]:
class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, crossover_rate, elitism_count):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
    
    def init_population(self, chromosome_len):
        '''
        creates the population with a predifined size and also chromosome len
        '''
        population = Population(self.population_size, chromosome_len)
        return population

    def calculate_fitness(self, individual, cities):
        '''
        count the fitness for each of the individual gens
        '''
        route = Route(individual, cities)
        fitness = 1 / route.get_distance()
        
        individual.fitness = fitness
        return fitness

    def eval_population(self, population, cities):
        population_fitness = 0
        
        # Loop over the population evaulating individuals and summing population fitness
        for i, individual in enumerate(population.individuals):
            population_fitness += self.calculate_fitness(individual, cities)
        average_fitness = population_fitness / population.size
        population.population_fitness = average_fitness
        
    def is_termination_condition_met(self, generations_count, max_generations):
        return generations_count > max_generations

    def select_parent(self, population):
        # Get individuals
        individuals = population.individuals
        
        # Spin roulette wheel
        population_fitness = population.population_fitness
        roulette_wheel_position = random.random() * population_fitness
        
        # Find parent
        spin_wheel = 0
        for individual in individuals:
            spin_wheel += individual.fitness
            if spin_wheel >= roulette_wheel_position:
                return individual
        
        # Return the last one if none of them matched
        return individuals[-1]
    
    def crossover_population(self, population, chromosome_len):
        new_population = Population(population.size, chromosome_len)
        
        for population_index in range(population.size):
            # Get parent 1
            parent1 = population.get_fittest(population_index)
            
            # Apply crossover to this individual
            if self.crossover_rate > random.random() and population_index > self.elitism_count:
                # Find parent 2 with tournament selection
                parent2 = self.select_parent(population)
                
                # Create blank offspring chromosome
                offspring_chromosome = [-1] * parent1.chromosome_len
                offspring = Individual(parent1.chromosome_len)
                offspring.chromosome = offspring_chromosome
                
                # Get subset of of parent chromosome
                substr_pos1 = int(random.random() * parent1.chromosome_len)
                substr_pos2 = int(random.random() * parent1.chromosome_len)
                
                # Make the samller the start and the larger the end
                start_substr = min(substr_pos1, substr_pos2)
                end_substr = max(substr_pos1, substr_pos2)
                
                # Loop and add the sub tour from parent1 to our child
                for i in range(start_substr, end_substr):
                    offspring.set_gene(i, parent1.get_gene(i))
                
                # Loop through parent2's city tour
                for i in range(parent2.chromosome_len):
                    parent2_gene = i + end_substr
                    if parent2_gene >= parent2.chromosome_len:
                        parent2_gene -= parent2.chromosome_len
                    
                    # If offspring doesn't have the city add it
                    if not offspring.contains_gene(parent2.get_gene(parent2_gene)):
                        # Loop to find a spare position in the child's tour
                        for j in range(offspring.chromosome_len):
                            # Spare position found, add city
                            if offspring.get_gene(j) == -1:
                                offspring.set_gene(j, parent2.get_gene(parent2_gene))
                                break
                # Add child
                new_population.set_individual(population_index, offspring)
            else:
                # Add individual to new population without applying crossover
                new_population.set_individual(population_index, parent1)

        return new_population
                
    
    def mutate_population(self, population, chromosome_len):
        pass


In [109]:
from math import sqrt

class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance_from(self, city):
        '''
        returns the difference of distance from a city
        '''
        deltaX = pow(city.x - self.x, 2)
        deltaY = pow(city.y - self.y, 2)
        distance = sqrt(abs(deltaX + deltaY))
        return distance

    def __str__(self):
        return print('x: {}, y: {}'.format(self.x, self.y))

In [119]:
class Route:
    def __init__(self, individual, cities):
        # Get individual's chromosome
        chromosome = individual.chromosome
        self.distance = 0
        
        # Create route
        self.route = [City] * len(cities)
        for gene_index in range(len(chromosome)):
            self.route[gene_index] = cities[chromosome[gene_index]]
    
    def get_distance(self):
        if self.distance > 0:
            return self.distance
        
        # Loop over cities in route and calculate route distance
        total_distance = 0
        for city_index in range(len(self.route)):
            if city_index + 1 < len(self.route):
                total_distance += self.route[city_index].distance_from(self.route[city_index + 1])
        
        total_distance += self.route[-1].distance_from(self.route[0])
        self.distance = total_distance
        return total_distance
    
        

In [140]:
class TravellingSalesmanProblem:
    def __init__(self):
        max_generations = 3000
        num_cities = 100
        
        # Create cities at random position
        x_pos = np.random.randint(100, size=num_cities)
        y_pos = np.random.randint(100, size=num_cities)
        
        cities = [City] * num_cities # np.empty((num_cities,))
        for i, (x, y) in enumerate(list(zip(x_pos, y_pos))):
            cities[i] = City(x, y)
        
        # Initial GA
        ga = GeneticAlgorithm(100, 0.001, 0.9, 2)
        chromosome_len = len(cities)
        population = ga.init_population(chromosome_len)
        
        # TODO: Evaluate populatino
        
        # Keep track of current generation
        generation = 1
        
        # Start evolution loop
        while not ga.is_termination_condition_met(generation, max_generations):
            # TODO: Print fittest individual from population
            route = Route(population.get_fittest(0), cities)
            print('best distance: {} iteration: {}'.format(route.get_distance(), generation))
            
            # TODO: Apply crossover
            population = ga.crossover_population(population, chromosome_len)
            
            # TODO: Apply mutation
            
            # TODO: Evaluate population
            ga.eval_population(population, cities)
            
            # Increment the current generation
            generation += 1
        
        # TODO: Display results

        print('stopped after {} generations'.format(max_generations))
        route = Route(population.get_fittest(0), cities)
        print('best distance: {}', route.get_distance())

TravellingSalesmanProblem()

best distance: 5571.560943645565 iteration: 1
best distance: 5434.420934617794 iteration: 2
best distance: 5204.307907091487 iteration: 3
best distance: 5090.240485332729 iteration: 4
best distance: 4988.52738020001 iteration: 5
best distance: 4876.017639869703 iteration: 6
best distance: 4790.5153448508245 iteration: 7
best distance: 4689.092849029616 iteration: 8
best distance: 4582.1554772430045 iteration: 9
best distance: 4388.28249667833 iteration: 10
best distance: 4332.334884405789 iteration: 11
best distance: 4208.33873444763 iteration: 12
best distance: 4151.510054814671 iteration: 13
best distance: 4099.223999368417 iteration: 14
best distance: 4048.402566421155 iteration: 15
best distance: 3939.314653305579 iteration: 16
best distance: 3841.6533158991338 iteration: 17
best distance: 3751.3219355391534 iteration: 18
best distance: 3683.350241775457 iteration: 19
best distance: 3613.526352397916 iteration: 20
best distance: 3569.1155335008907 iteration: 21
best distance: 3479.